# NMT

In [3]:
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
from tensorflow.python.layers import core as layers_core
import tensorflow as tf
import tensorflow.compat.v1 as tfc


c:\Users\Michael\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Michael\anaconda3\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sur

### Read data

In [4]:
enline, chline = None, None
enlines, chlines = [], []
sentence_length = 10
file_path = '../data/word/segmented_train_seg_by_word.txt'
pbar = tqdm()
i = 0
early_exist_line_num = 1000000
with open(file_path, 'r') as lines:
    for line in lines:
        i += 1
        if i % 2 == 1:
            enline = line.lower().strip('\n').split()
            continue          
        else:
            chline =[w for w in line.strip('\n').replace(' ','')]
        if len(enline) > sentence_length and len(chline) > sentence_length:
            continue    
        enlines.append(enline)
        chlines.append(chline)  
        if i % 100000 == 0:
            pbar.update(i)  
        if i == early_exist_line_num:
            break
     

3100000it [00:04, 840862.55it/s]

In [5]:
len(chlines)

301514

In [6]:
print(chlines[:4])
print(enlines[:4])

[['一', '对', '二', '总', '不', '是', '好', '事', '，'], ['一', '对', '二', '胜', '。'], ['一', '对', '五', '年', '没', '见', '过', '的', '姐', '妹', '一', '场', '激', '烈', '的', '争', '吵', '？'], ['一', '对', '五', '百', '诶', '。']]
[['fighting', 'two', 'against', 'one', 'is', 'never', 'ideal', ','], ['deuces', 'the', 'winner', '.'], ['an', 'incredibly', 'emotional', 'fight', 'between', '2', 'sisters', '？'], ['one', 'against', '500', '.']]


In [7]:
def addchar(word2id, id2word, word):
    id = len(word2id)
    word2id[word] = id
    id2word[id] = word

ch2id, id2ch, en2id, id2en = {}, {}, {},{}
most_common_k = 100000
specialchars = ['<eos>','<start>','<end>','<unk>','<pad>']

chwords, enwords = set(), set()
for ch in chlines: 
    for c in ch:
        chwords.add(c)
for en in enlines: 
    for e in en:
        enwords.add(e)

for word, _ in Counter(chwords).most_common(most_common_k):
    addchar(ch2id, id2ch, word)
    
for word, _ in Counter(enwords).most_common(most_common_k):
    addchar(en2id, id2en, word)
    
for one in specialchars:
    addchar(ch2id,id2ch,one)
    addchar(en2id,id2en,one)

In [8]:
len(en2id), len(ch2id)
# ch2id['<unk>']

(47029, 4771)

### Prepare data for english to chinese translation

In [9]:
dat_x_in, dat_y_in,dat_y_out =[], [], []
dat_x_len, dat_y_len = [], []
unknown_en_id = en2id['<unk>']
eos_en_id = en2id['<eos>']
# english sentences to id array
for en_sentence in enlines:
    id_en_sentence = [ en2id.get(w, unknown_en_id) for w in en_sentence ]
    id_en_sentence.append(eos_en_id) #mark end of sentence
    dat_x_in.append(id_en_sentence)
    dat_x_len.append(len(id_en_sentence))
    
# chinse sentences to id array
start_ch_id = ch2id['<start>']
end_ch_id = ch2id['<end>']
unknown_ch_id = ch2id['<unk>']
for ch_sentence in chlines:
    id_ch_sentence = [ ch2id.get(w, unknown_ch_id) for w in ch_sentence ]
    dat_y_in.append([start_ch_id] + id_ch_sentence)
    dat_y_out.append(id_ch_sentence + [end_ch_id])
    dat_y_len.append(len(id_ch_sentence) + 1)   

### Transform data

In [10]:
m = 100
[id2en[i] for i in dat_x_in[m]], [id2ch[i] for i in dat_y_in[m]]

(['one', 'pair', 'of', 'elongated', 'canines', '.', '<eos>'],
 ['<start>', '一', '对', '延', '长', '犬', '齿', '。'])

In [11]:
# build input and lable
data_x_in, data_y = [],[]

for ch_sentence, en_sentence in zip(chlines, enlines):
    ch_id_sen = [ch2id.get(i, unknown_ch_id) for i in ch_sentence]
    data_x_in.append(ch_id_sen)
    data_y.append(0)

    en_id_sen = [en2id.get(i, unknown_en_id) for i in en_sentence]
    data_x_in.append(en_id_sen)
    data_y.append(1)

data_x_in = tf.keras.preprocessing.sequence.pad_sequences(data_x_in, padding='post', value=en2id['<pad>'])
data_y = np.asarray(data_y)
    

3100000it [00:20, 840862.55it/s]

### Buil model

In [13]:
batch_size = 256
embedding_size = 100
vocabulary_size = len(en2id)
num_units = 50

In [16]:
embedding_size = 512   #word embedding vector length
num_units = 512        #hidden layer size
batch_size = 128
max_grad = 1.0         #L2裁剪的c
dropout = 0.2
src_vocab_size = len(en2id) #vocabulary
target_vocat_size = len(ch2id)
seq_max_len = sentence_length + 1
maximum_iterations = 10  #预测长度超过10时终结

In [20]:
tfc.reset_default_graph()
tfc.disable_eager_execution()
config = tfc.ConfigProto(log_device_placement =True, allow_soft_placement = True)
config.gpu_options.allow_growth = True
session = tfc.Session(config = config)

with tfc.device('/gpu:1'):
    initializer =tfc.random_uniform_initializer(-0.08, 0.08)
    tfc.get_variable_scope().set_initializer(initializer)

    x = tfc.placeholder('int32', [None, None])
    y = tfc.placeholder('int32', [None])
    x_len = tfc.placeholder('int32', [None]) # control sequence lenth for RNN
    y_len = tfc.placeholder('int32', [None]) # control sequence lenth for RNN
    learning_rate = tfc.placeholder(tfc.float32, shape=[])

    # embedding
    embedding_encoder = tfc.get_variable('embedding_encoder', [vocabulary_size, embedding_size], dtype=tfc.float32)
    embedding_decoder = tfc.get_variable('embedding_decoder',[target_vocat_size, embedding_size], dtype=tfc.float32)
    encoder_embedding_inp = tfc.nn.embedding_lookup(embedding_encoder, x)
    decoder_embedding_inp = tfc.nn.embedding_lookup(embedding_decoder, x)

# encoder

    # build rnn cell
    encoder_cell = tfc.nn.rnn_cell.BasicLSTMCell(num_units)
    encoder_outputs, encoder_states = tfc.nn.dynamic_rnn(encoder_cell, decoder_embedding_inp, sequence_length= x_len, time_major=False, dtype=tfc.float32)

# decoder
    batch_size_in = tf.shape(x)[0]
    projection_layer = layers_core.Dense(len(ch2id), use_bias=False)
    
    # dynamic decoding
    with tfc.variable_scope("decode_layer"):
        decoder_cell = tfc.nn.rnn_cell.BasicLSTMCell(num_units)
        
        sampler = tfa.seq2seq.TrainingSampler(time_major=False)
        sampler.initialize(decoder_embedding_inp, y_len)
        
        decoder = tfa.seq2seq.BasicDecoder(decoder_cell, sampler, output_layer= projection_layer)

    # loss
    model_logistic = tfc.layers.dense(encoder_states[0], 1)
    model_prediction = tfc.nn.sigmoid(model_logistic)
    loss = tfc.nn.sigmoid_cross_entropy_with_logits(labels=tfc.cast(y, tfc.float32), logits=tfc.reshape(model_logistic, (-1,)))
    loss1 = tfc.reduce_mean(loss)
    optimizer = tfc.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

Device mapping: no known devices.


C:\Users\Michael\AppData\Local\Temp\ipykernel_16076\2504026095.py:26: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  encoder_cell = tfc.nn.rnn_cell.BasicLSTMCell(num_units)
C:\Users\Michael\AppData\Local\Temp\ipykernel_16076\2504026095.py:35: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  decoder_cell = tfc.nn.rnn_cell.BasicLSTMCell(num_units)


TypeError: type of argument "output_layer" must be one of (keras.src.engine.base_layer.Layer, NoneType); got tensorflow.python.keras.legacy_tf_layers.core.Dense instead

In [ ]:
session.run(tf.global_variables_initializer())


### Training

In [ ]:
def data_generate(data_x_in, data_y, batch_size = 1000):
    for i in range(0, len(data_x_in), batch_size):
        if i + batch_size < len(data_x_in):
            yield data_x_in[i: i + batch_size], data_y[i: i + batch_size]
        else:
            yield data_x_in[i: len(data_x_in)], data_y[i: len(data_x_in)]

In [ ]:
loss = []
begging_learning_rate = 0.1
gen = data_generate(data_x_in, data_y, batch_size)